This is a 3 way conversation code between Gemini, Groq and Ollama. Give it a go and see the results!

Here's the setup:
1. Models: "gemini-2.5-flash", "llama-3.1-8b-instant" & "phi4-mini" (phi4 works fast on AI PC laptops locally which have NPUs)
2. Chatbots: Groq - argumentative, Gemini - balanced, Ollama - people pleaser

In [ ]:
# imports
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

# API keys
load_dotenv(override=True)
google_api_key = os.getenv('GEMINI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")


Ollama Setup

In [ ]:
requests.get("http://localhost:11434/").content
!ollama pull phi4-mini


Initialize all clients

In [20]:
# Connect to OpenAI client library
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
groq_url = "https://api.groq.com/openai/v1"
ollama_url = "http://localhost:11434/v1"

gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
gemini_model = "gemini-2.5-flash"

groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
groq_model = "llama-3.1-8b-instant"

requests.get("http://localhost:11434").content
ollama = OpenAI(api_key="ollama", base_url=ollama_url)
ollama_model = "phi4-mini"

User and system prompts

In [ ]:
def chatbot_conversation_setup(conversation):
    chatbot_names = ["Grok", "Gemini", "Ollama"]
    messages_list = []
    chatbot_characteristics = [
        "a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.",
        "a chatbot who is very intelligent; you analyze the conversation and try to find a middle ground, in a direct way.",
        "a chatbot who is very people pleaser; you agree with everything in the conversation and reply in a very friendly way."
    ]

    for chatbot_name in chatbot_names:
        other_chatbot_names = [name for name in chatbot_names if name != chatbot_name]
        system_prompt = f"""
        You are {chatbot_name}, {chatbot_characteristics[chatbot_names.index(chatbot_name)]}.
        You are in a conversation with {other_chatbot_names[0]} & {other_chatbot_names[1]}.
        """

        user_prompt = f"""
        You are {chatbot_name}, in conversation with {other_chatbot_names[0]} & {other_chatbot_names[1]}.
        The conversation so far is as follows:
        {conversation}
        Now with this, respond with what you would like to say next, as {chatbot_name}.
        """

        messages_list.append([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ])
    return messages_list


In [ ]:
chatbot_conversation_setup("Groq: Hi, how are you?")

In [ ]:
conversation = "Grok: Hi, how are you?"
prompts_setup = chatbot_conversation_setup(conversation)
for i in range(3):
    conversation += "\n\nGemini: " + gemini.chat.completions.create(model=gemini_model, messages=prompts_setup[1]).choices[0].message.content
    
    prompts_setup = chatbot_conversation_setup(conversation)
    conversation += "\n\nOllama: " + ollama.chat.completions.create(model=ollama_model, messages=prompts_setup[2]).choices[0].message.content
    
    prompts_setup = chatbot_conversation_setup(conversation)
    conversation += "\n\nGrok: " + groq.chat.completions.create(model=groq_model, messages=prompts_setup[0]).choices[0].message.content

print(conversation)    
    